In [29]:
import asyncio
import httpx
from bs4 import BeautifulSoup
from tqdm.asyncio import tqdm_asyncio
from pathlib import Path
import time

In [30]:
BASE_URL = "https://www.blacktowhite.net"
HEADERS = {"User-Agent": "Mozilla/5.0"}


async def fetch(client, url):
    response = await client.get(url, follow_redirects=True)
    return response.text


async def fetch_video_src(client, video_page_url):
    response = await client.get(video_page_url)
    soup = BeautifulSoup(response.text, "html.parser")
    source = soup.find("source")
    if source and source.get("src"):
        return source["src"]
    return None


async def get_video_links_from_page(client, page, category):
    url = f"{BASE_URL}/{category}/page-{page}?order=view_count&direction=desc&type=video&newer_than=all"
    html = await fetch(client, url)
    soup = BeautifulSoup(html, "html.parser")
    tasks = []

    for a_tag in soup.find_all("a"):
        if a_tag.has_attr("data-src"):
            video_page_url = BASE_URL + a_tag["data-src"]
            tasks.append(fetch_video_src(client, video_page_url))

    return await asyncio.gather(*tasks)


async def main(category):
    videos = []
    async with httpx.AsyncClient(headers=HEADERS, timeout=30) as client:
        tasks = [
            get_video_links_from_page(client, page, category) for page in range(1, 10)
        ]
        results = await asyncio.gather(*tasks)
        for result in results:
            videos.extend(filter(None, result))  # Filter out any Nones

    print(f"Total videos found: {len(videos)}")
    return videos


for category in ["cuckold-vodeos", "blowjob-videos", "fucking-videos", "videos"]:
    print(category)
    time.sleep(5)
    videos = await main(category)
    html = Path("template.html").read_text()
    with open(f"{category}.html", "w", encoding="utf-8") as f:
        f.write(
            html.replace(
                "<--videos-->",
                " ".join(
                    [
                        f"""
    <div class="video-tile" data-video-url="{BASE_URL}{url}">
      <a href="{BASE_URL}{url}" target="_blank">
        <video src="{BASE_URL}{url}" preload="metadata" muted></video>
        <span class="duration">Loading...</span>
      </a>
    </div>"""
                        for url in videos
                    ]
                ),
            )
        )


cuckold-vodeos


Task exception was never retrieved
future: <Task finished name='Task-292' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at c:\Users\admin\.python-venvs\py13\Lib\site-packages\tqdm\asyncio.py:75> exception=ConnectError('All connection attempts failed')>
Traceback (most recent call last):
  File "c:\Users\admin\.python-venvs\py13\Lib\site-packages\httpx\_transports\default.py", line 101, in map_httpcore_exceptions
    yield
  File "c:\Users\admin\.python-venvs\py13\Lib\site-packages\httpx\_transports\default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\admin\.python-venvs\py13\Lib\site-packages\httpcore\_async\connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "c:\Users\admin\.python-venvs\py13\Lib\site-packages\httpcore\_async\connection_pool.py", line 236, in handle_async_request
    response = await connection.han

Total videos found: 216
blowjob-videos
Total videos found: 216
fucking-videos
Total videos found: 216
videos
Total videos found: 216
